# Lab Assignment Three: Extending Logistic Regression 

Arely Alcantara, Emily Fashenpour

## 1. Preparation and Overview

### 1.1 Business Case

The dataset that we selected is titled "Wine Quality" and it looks at a total of 6497 samples of both red and white wines. The dataset looks at 11 attributes of each wine sample such as the acidity, amount of alcohol, and density just to name a few. Each wine sample was also given a quality value that rates how good or bad that wine is on a numerical scale between 0 and 10. Given a new wine company entering the market, we hope to be able to classify and find what its quality is relative to the other existing samples in our dataset. Third parties interested  in this information might be new companies or people trying to enter the wine market and seeing how their creation relates to existing wines in the market and how its quality might be in relation to existing wineries.

We could essentially deploy this solution and charge a small fee to those companies wanting to find out what their wine quality is before they launch themselves into the market. In analyzing this dataset, we hope to help different wineries obtain a quality value for their wine creation and take into account the same attributes that our dataset uses.

Dataset URL: https://www.kaggle.com/danielpanizzo/wine-quality

Classification task: Wine quality on scale of 0 to 10

### 1.2 Data Preparation 

### 1.3 Divide data into training and training sets

## 2. Modeling

### 2.1 Implementation of logistic regression

### 2.2 One-versus-all logistic regression classifier

### 2.3 Train custom classifier

### 2.4 Compare custom results and scikit-learn

## 3. Deployment

## 4. Exceptional Work